# Integrantes do grupo
Bruno dos Santos, **NUSP 10786170**

Henrique de S. Q. dos Santos, **NUSP 10819029**

Paulo H. da Silva, **NUSP 10734515**

Witor M. A. de Oliveira, **NUSP 10692190**

# Descrição do Problema
Abaixo, definimos a função objetivo, as restrições e variáveis do modelo.

Função objetivo: $\min z = \sum_{(i,j) \in G}^{}(D_{ij} \times p_{ij})$

Restrições:

- (1)&nbsp; $\sum_{i=1}^{n}p_{ij} = 1, j = 1,...,n$
- (2)&nbsp; $\sum_{j=1}^{n}p_{ji} = 1, i = 1,...,n$
- (3)&nbsp; $\sum_{i,j \in g} p_{ij} = |g|$
- (4)&nbsp; $p_{ij} + p_{ji} \leq 1$
- (5)&nbsp; $p_{ij} \in \{0,1\}, \forall i,j \in G$
- (6)&nbsp; $|g| > 2,  \forall G$
- (7)&nbsp; $D_{ij} = M, i=j, j = 1,...,n$

Onde:

- $D_{ij}$ é o custo (dado pela distância euclideana) de visitar a galáxia $j$ a partir de $i$
- $p_{ij}$ = 1, se a aresta $(i,j)$ for escolhida como caminho;
- $p_{ij}$ = 0, caso contrário;
- $g$ é um subconjunto do conjunto de galáxias $G$;
- $|g|$ é o número de galáxias do subconjunto $g$;
- $M$ é um valor inteiro suficientemente grande.

E:

- A restrição (1) indica que só podemos chegar em $j$ a partir de uma aresta, ou seja, nunca visitaremos $j$ mais de uma vez;
- A restrição (2) indica que só podemos sair de uma cidade $j$ uma vez;
- A restrição (3) garante que todos os nós tem que ser visitados;
- A restrição (4) determina que uma aresta é percorrida no máximo vez;
- A restrição (5) determina que a variável $p$ seja binária, ou seja, só assume o valor 0 ou 1;
- A restrição (6) determina que deve existir pelo menos 3 galáxias em $G$;

# Instalação do PuLP
Esse foi o software utilizado para resolução do problema modelado. A documentação dele está presente **[aqui](https://coin-or.github.io/pulp/)**.

In [ ]:
pip install PuLP

# Importando os módulos necessários para executar a implementação

In [ ]:
import math
import pulp as pl
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances

# Início da implementação

In [ ]:
# Coordenadas das N galáxias. Para adicionar mais uma galáxia, basta
# adicionar mais um vetor de coordenadas [x,y].
G = np.asarray([[4,5], [7,4], [6,2], [1,3], [2,0]])

# Calculando as distâncias euclidianas entre cada galáxias. Usaremos
# esse resultado como sendo o peso de se viajar de uma galáxia 'i'
# para uma galáxia 'j'.
dist = euclidean_distances(G,G)

numGalaxias = len(G)

print(dist)

[[0.         3.16227766 3.60555128 3.60555128 5.38516481]
 [3.16227766 0.         2.23606798 6.08276253 6.40312424]
 [3.60555128 2.23606798 0.         5.09901951 4.47213595]
 [3.60555128 6.08276253 5.09901951 0.         3.16227766]
 [5.38516481 6.40312424 4.47213595 3.16227766 0.        ]]


In [ ]:
# Definindo o nosso problema
prob = pl.LpProblem("EnigmaDasGaláxias", pl.LpMinimize) 

obj = [] # Variável que armazenará a função objetivo
aux = {} # Dicionário auxiliar
aux1 = {} # Dicionário auxiliar
aux2 = {} # Dicionário auxiliar
aux3 = [] # Vetor auxiliar

p = {} # Dicionário que armazenará nossas variáveis binárias pij

for i in range(0, numGalaxias):
  for j in range(0, numGalaxias):
      if i != j:
        #   Já garante a restrição 5 do problema
        p[i,j] = pl.LpVariable("P"+str(i)+str(j), cat='Binary') 

# Definindo a restrição 1
for i in range(numGalaxias):
    aux[i] = pl.LpAffineExpression([(p[i, j], 1) for j in range(numGalaxias) if i!=j])
    # Adicionando a restrição ao modelo
    prob += pl.LpConstraint(aux[i], 0, "Restrição 1:" + str(i), 1)

# Definindo a restrição 2
for i in range(numGalaxias):
  aux1[i] = pl.LpAffineExpression([(p[j, i], 1) for j in range(numGalaxias) if i!=j])
  # Adicionando a restrição ao modelo
  prob += pl.LpConstraint(aux1[i], 0, "soma de cada coluna eh 1 - coluna "  + str(i), 1)

# Definindo a restrição 3
k = 0 # Auxiliar pra percorrer o aux2

for i in range(numGalaxias):
  for j in range(i, numGalaxias):
    if i != j: 
      
      # Auxiliar para montar a expressao da combinação linear para a restrição
      aux2[k] = pl.LpAffineExpression([(p[i, j], 1), (p[j, i], 1)])
      # Adicionando a restrição ao modelo
      prob += pl.LpConstraint(aux2[k], -1, "restrição de ida e vinda" + str(k), 1)

      k = k + 1

    else:
      continue

# Definindo a restrição 4 - equivale a restrição 3 do trampo 
for i in range(numGalaxias):
  aux3 += pl.LpAffineExpression([(p[i, j], 1) for j in range(numGalaxias) if i!=j])

# Adicionando restrição ao modelo
prob +=  pl.LpConstraint(aux3, 0, "vistia todos nós obrigatóriamente", numGalaxias)

# A restrição 6 do problema foi garantida ao evitar que os laços
# fossem executados quando i == j

# Definindo a função objetivo
for i in range(numGalaxias):
  obj += pl.LpAffineExpression([(p[i, j], dist[i][j]) for j in range(numGalaxias) if i!=j])

# Adicionando a função objetivo ao modelo
prob += obj

status = prob.solve()
print("Tempo de execução: ", prob.solutionTime)

result = 0

print("Caminho percorrido [(ida) -> (chegada)]")
for i in range(0, numGalaxias):
  for j in range(0, numGalaxias):
      if (i!=j):
        if (pl.value(p[i,j])):
            result += dist[i][j]
            print(str(i) + " -> " + str(j))

print("Custo do caminho mínimo: ", result)

Tempo de execução:  0.007536460999999939
Caminho percorrido [(ida) -> (chegada)]
0 -> 1
1 -> 2
2 -> 4
3 -> 0
4 -> 3
Custo do caminho mínimo:  16.638310528300117
